## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [4]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [96]:
convention_db = sqlite3.connect("/users/kayan/Desktop/USD/ADS_509/2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [97]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text,party
                            FROM conventions;
                            ''')

for row in query_results:
    text, party = row
    convention_data.append([text, party])

# Close the database connection
convention_db.close()


Let's look at some random entries and see if they look right. 

In [98]:
random.choices(convention_data,k=10)

[['Congratulations, you’re now citizens of the United States of America. On behalf of the Department of Homeland Security, it is my honor to call you my fellow Americans. Mr. President, I want to again commend you for your dedication to the rule of law and for restoring integrity to our immigration system. Thank you for hosting such a patriotic celebration here at the White House today.',
  'Republican'],
 ['I’m pleased to announce that Vice President Joe Biden has officially been nominated by the Democratic Party as our candidate for President of the United States.',
  'Democratic'],
 ['Every generation before us has had to fight for what they believe in and it’s just our turn now. Jack G.: ( 01:02:03 )  I was proud when I saw the demonstrations that were going on across the country.',
  'Democratic'],
 ['Joe Biden has selected Kamala Harris as his running mate.', 'Democratic'],
 ['American history tells us that’s been in our darkest moments that we’ve made our greatest progress that 

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [99]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2891 as features in the model.


In [100]:
def conv_features(text,feature_words) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * feature_words: the *feature words* that we're considering. A word 
            in `text` must be in feature_words in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `feature_words`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `feature_words` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    
    ret_dict = {word.strip().lower(): True for word in text.strip().lower().split() if word.strip().lower() in feature_words}
    return ret_dict

In [106]:
conv_features("donald is the president",feature_words)

{'is': True, 'the': True, 'president': True}

In [101]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president", feature_words)==
                     {'donald':True,'president':True})
assert(conv_features("some people in america are citizens",feature_words)==
                     {'people':True,'america':True,'citizens':True})

AssertionError: 

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [102]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [103]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [104]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.432


In [105]:
classifier.show_most_informative_features(25)

Most Informative Features
             enforcement = True           Republ : Democr =     27.5 : 1.0
                   votes = True           Democr : Republ =     21.6 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                 destroy = True           Republ : Democr =     17.1 : 1.0
                supports = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     15.9 : 1.0
                preserve = True           Republ : Democr =     15.1 : 1.0
                  signed = True           Republ : Democr =     15.1 : 1.0
                freedoms = True           Republ : Democr =     14.0 : 1.0
                freedom, = True           Republ : Democr =     13.4 : 1.0
                 private = True           Republ : Democr =     11.9 : 1.0
                freedom. = True           Republ : Democr =     11.5 : 1.0
                    drug = True           Republ : Democr =     10.9 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations
The accuracy of nltk is 0.432, which means less than half of the instaces are classified correctly.




## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [49]:
cong_db = sqlite3.connect("/users/kayan/Desktop/USD/ADS_509/congressional_data.db")
cong_cur = cong_db.cursor()

In [50]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [52]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.
for row in results:
    candidate, party, tweet_text = row
    features = conv_features(tweet_text, feature_words)
    tweet_data.append([features, party])

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [53]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [54]:

for tweet, party in tweet_data_sample :
    estimated_party = 'Gotta fill this in'
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: {}
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: {}
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: {}
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: {}
Actual party is Republican and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: {}
Actual party is Republican and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: {}
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: {}
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: {}
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: {}
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: {}
Actual party is Democratic and our classifer says G

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [55]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = "Gotta fill this in"
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [56]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 0,
                          'Democratic': 0,
                          'Gotta fill this in': 4278}),
             'Democratic': defaultdict(int,
                         {'Republican': 0,
                          'Democratic': 0,
                          'Gotta fill this in': 5724})})

### Reflections

The republican has 278 results and democratic has 5724 results. 